In [128]:
import pandas as pd

In [129]:
# data manipulation and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for saving the pipeline
import joblib

# from Scikit-learn
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Binarizer

# from feature-engine
from feature_engine.imputation import (
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer,
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder,
)

from feature_engine.transformation import LogTransformer

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import MultiLabelBinarizer

# import preprocessor as pp


from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler

#XGboost classifier
from xgboost import XGBClassifier


#import PCA
from sklearn.decomposition import PCA

#one hot encoding
from sklearn.preprocessing import OneHotEncoder

In [130]:
# load dataset
df = pd.read_csv('../output/saida.csv')

# rows and columns of the data
print(df.shape)

# visualise the dataset
df.head()

(484381, 35)


,id_form,id_cliente,data,sistema_operacional,navegador,localizacao,locale_x,cidade_x,estado_x,qtde_lavagem_semana,...,status_pagamento,tipo_pagamento,valor_total,valor_desconto,frete_gratis,cidade_y,estado_y,locale_y,utm_source_y,target
0,c7d453c9c01223a42968d9fe136047,d2620a3e6f00a6dc6e88,2022-01-03 15:42:22,iOS,Safari,"São Paulo, Sao Paulo",BR,SÃO PAULO,SAO PAULO,4,...,Entregue,CARTÃO,124.03,0.0,False,São Paulo,SP,BR,NaN,1.0
1,72830da153824af486be15cd48b3f3,9a48802d8290d19471f3,2022-01-04 08:39:53,Windows,Chrome,"Ribeirão Preto, Sao Paulo",BR,RIBEIRÃO PRETO,SAO PAULO,7,...,Cancelado,PIX,282.22,0.0,False,Jundiaí,SP,BR,NaN,1.0
2,72830da153824af486be15cd48b3f3,9a48802d8290d19471f3,2022-01-04 08:39:53,Windows,Chrome,"Ribeirão Preto, Sao Paulo",BR,RIBEIRÃO PRETO,SAO PAULO,7,...,Cancelado,PIX,282.22,0.0,False,Jundiaí,SP,BR,NaN,1.0
3,d101fc0075dac29db1168cfc61ad44,c3344fad450ee424700d,2022-01-04 19:34:05,Android,Chrome,"São Paulo, Sao Paulo",BR,SÃO PAULO,SAO PAULO,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0957fb9d7be36b4d079b6582bb6785,8105e34124e90300d8b5,2022-01-05 18:29:41,Android,Chrome,"Manchester, England",BR,MANCHESTER,ENGLAND,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# df.drop(['id_form','status_pagamento', 'tipo_pagamento', 'valor_total',
#        'valor_desconto', 'frete_gratis', 'cidade_y', 'estado_y', 'locale_y',
#        'utm_source_y','id_pedido','data','data_pedido'], inplace = True, axis=1)

In [132]:
#convert columns caracteristic and procedimentos to list
df['caracteristica'] = df['caracteristica'].apply(lambda x: x.replace(' ',''))
df['caracteristica'] = df['caracteristica'].apply(lambda x: x.split(','))


In [133]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('caracteristica')),
                index=df.index,
                columns=mlb.classes_))

# Split the data into train and test set

In [134]:
# Let's separate into train and test set
# Remember to set the seed (random_state for this sklearn function)

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['target'], axis=1), # predictive variables
    df['target'], # target
    test_size=0.2, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)

X_train.shape, X_test.shape

((387504, 44), (96877, 44))

In [135]:
X_train.tail(20)

,id_form,id_cliente,data,sistema_operacional,navegador,localizacao,locale_x,cidade_x,estado_x,qtde_lavagem_semana,...,Característica10,Característica2,Característica3,Característica4,Característica5,Característica6,Característica7,Característica8,Característica9,N/C
17089,269bd07c50220855cc090b16526b8a,fe0ad6d49f61c62575fa,2022-01-05 19:53:32,Android,Chrome,"Itaquirai, Mato Grosso do Sul",BR,ITAQUIRAI,MATO GROSSO DO SUL,1,...,0,0,1,0,0,1,0,1,0,0
80186,136afbe0975e61031a56837282822d,8f0801e571ad985fcee2,2022-01-24 22:41:27,Android,Chrome,"Piui, Minas Gerais",BR,PIUI,MINAS GERAIS,5,...,0,0,0,0,0,0,0,1,0,0
314764,c5040d9526ddcfcccd944ef6a38a88,2fadcbc671bd4bec8078,2022-04-30 08:18:35,iOS,NaN,"Vila Velha, Espirito Santo",BR,VILA VELHA,ESPIRITO SANTO,3,...,0,0,0,0,0,0,0,0,0,0
170584,51229569559d6cb93237e0c42b4ccd,2ca7ba7e35513b5e0890,2022-02-27 16:14:43,Android,Chrome,"Rio do Sul, Santa Catarina",BR,RIO DO SUL,SANTA CATARINA,2,...,0,0,1,0,0,0,0,1,0,0
310744,4fab4ed9cfbc3605f79e201dba8bea,1d786782987b9c7037cf,2022-04-28 15:47:50,Android,Chrome,"Nova Iguaçu, Rio de Janeiro",BR,NOVA IGUAÇU,RIO DE JANEIRO,1,...,0,0,1,0,0,0,0,0,0,0
212038,64b4d13db5682a5ec4fb0aa65866fd,8d91c6bac7075c96938b,2022-03-22 21:40:57,iOS,NaN,"Porto Alegre, Rio Grande do Sul",BR,PORTO ALEGRE,RIO GRANDE DO SUL,4,...,0,0,1,0,0,0,0,0,0,0
211543,db678aa14d194349fef7d80dbf90d8,60d697574b32d09deddd,2022-03-22 19:36:01,iOS,Safari,"João Pessoa, Paraíba",BR,JOÃO PESSOA,PARAÍBA,1,...,0,1,0,0,0,0,0,0,0,0
374564,40d78a18f7a519a8e39168bca383d8,abe968728bbe53a38f12,2022-05-22 17:43:31,Android,Chrome,"Porto Alegre, Rio Grande do Sul",BR,PORTO ALEGRE,RIO GRANDE DO SUL,5,...,0,0,0,0,0,0,0,1,1,0
448242,b98435a4a17fd45f622738ec6c4ee5,ec925d86b78d4537a8f1,2022-06-20 22:59:07,Android,Chrome,"Santo André, Sao Paulo",BR,SANTO ANDRÉ,SAO PAULO,1,...,0,0,1,1,0,0,0,0,0,0
86293,bca380b787aca9473e804781babfe0,48c0b8abcc33843ae0e5,2022-01-26 23:20:30,Android,Chrome,"São Paulo, Sao Paulo",BR,SÃO PAULO,SAO PAULO,5,...,0,0,1,0,0,0,0,0,0,0


## Target

In [136]:
y_train.fillna(0, inplace=True)
y_test.fillna(0, inplace=True)

## Configurations

In [137]:
# categorical variables with NA in train set
CATEGORICAL_VARS_WITH_NA_MISSING = [
 'cidade_x',
 'estado_x',
 'efeitos_desejados'
]

# categorical variables with NA in train set
CATEGORICAL_VARS_WITH_NA_FREQUENT = [
 'navegador',
 'utm_source_x',

]

# numerical variables
NUMERICAL_VARS = ['qtde_lavagem_semana']



# this variable is to calculate the temporal variable,
# can be dropped afterwards
DROP_FEATURES = ["procedimentos","efeitos_desejados",'id_cliente','localizacao','id_form','status_pagamento', 'tipo_pagamento', 'valor_total',
       'valor_desconto', 'frete_gratis', 'cidade_y', 'estado_y', 'locale_y',
       'utm_source_y','id_pedido','data','data_pedido']


# variables to binarize
BINARIZE_VARS = ['procedimentos','efeitos_desejados']


# categorical variables to encode
CATEGORICAL_VARS = [
                     'sistema_operacional',
                     'navegador',
                     'locale_x',
                     'cidade_x',
                     'estado_x',
                     'utm_source_x',
                     'tipo_cabelo',
                     'comprimento',
                     'tipo_fios',
                     'tempo_procedimento',
                     'dieta',
                     'atividade_fisica',
                     'frequencia_estresse',
                     'faixa_etaria',
                     'fragancia',
                    'efeitos_desejados'
]


#feature_selected = CATEGORICAL_VARS + NUMERICAL_VARS
# variable mappings
frequencia_estresse = {'Nunca': 1, 'Regularmente': 4, 'Quase sempre': 3,
                        'Uma vez por mês': 2,  'Missing': 0, 'NA': 0}


under = RandomUnderSampler(sampling_strategy=0.5)
X_train, y_train = under.fit_resample(X_train, y_train)
X_test, y_test = under.fit_resample(X_test, y_test)

C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\venv\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\venv\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [138]:
price_pipe = Pipeline([

    # ===== IMPUTATION =====
    # impute categorical variables with string missing
    ('missing_imputation', CategoricalImputer(
        imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)),

    ('frequent_imputation', CategoricalImputer(
        imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)),

    # add missing indicator
    # ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA)),
    #
    # # impute numerical variables with the mean
    # ('mean_imputation', MeanMedianImputer(
    #     imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA
    # )),


    # == TEMPORAL VARIABLES ====
    # ('elapsed_time', pp.TemporalVariableTransformer(
    #     variables=TEMPORAL_VARS, reference_variable=REF_VAR)),
    #
    # ('drop_features', DropFeatures(features_to_drop=[REF_VAR])),



    # # ==== VARIABLE TRANSFORMATION =====
    # ('log', LogTransformer(variables=NUMERICALS_LOG_VARS)),

#     ('yeojohnson', YeoJohnsonTransformer(variables=NUMERICALS_YEO_VARS)),
#
#     ('binarizer', SklearnTransformerWrapper(
#         transformer=Binarizer(threshold=0), variables=BINARIZE_VARS)),

    # ('multilabel', SklearnTransformerWrapper(
    #     transformer=MultiLabelBinarizer(), variables=BINARIZE_VARS)),

    # # === mappers ===
    # ('mapper_qual', pp.Mapper(
    #     variables=QUAL_VARS, mappings=QUAL_MAPPINGS)),
    #
    # ('mapper_exposure', pp.Mapper(
    #     variables=EXPOSURE_VARS, mappings=EXPOSURE_MAPPINGS)),
    #
    # ('mapper_finish', pp.Mapper(
    #     variables=FINISH_VARS, mappings=FINISH_MAPPINGS)),
    #
    # ('mapper_garage', pp.Mapper(
    #     variables=GARAGE_VARS, mappings=GARAGE_MAPPINGS)),

#     ('mapper_fence', pp.Mapper(
#         variables=FENCE_VARS, mappings=FENCE_MAPPINGS)),

    #

    # == CATEGORICAL ENCODING
    ('rare_label_encoder', RareLabelEncoder(
        tol=0.01, n_categories=1, variables=CATEGORICAL_VARS
    )),
#
    # encode categorical and discrete variables using the target mean
    ('categorical_encoder', OrdinalEncoder(
        encoding_method='ordered', variables=CATEGORICAL_VARS)),



#
#
#     ('scaler', MinMaxScaler()),
# #     ('selector', SelectFromModel(Lasso(alpha=0.001, random_state=0))),
#     ('Lasso', Lasso(alpha=0.001, random_state=0)),


    #drop features
    ('drop_features', DropFeatures(features_to_drop=DROP_FEATURES)),

    #PCA
    ('pca', PCA(n_components=0.95)),



    #MODEL Xgboost
    ('xgboost', XGBClassifier(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        objective='binary:logistic',
        booster='gbtree',
        n_jobs=-1,
        random_state=0,
        verbose=True
    )),


])



In [139]:
X_train

,id_form,id_cliente,data,sistema_operacional,navegador,localizacao,locale_x,cidade_x,estado_x,qtde_lavagem_semana,...,Característica10,Característica2,Característica3,Característica4,Característica5,Característica6,Característica7,Característica8,Característica9,N/C
0,6d768849518ef901b121196cb158c0,b15d5448c08b5cb6ee98,2022-03-12 07:48:03,Android,Chrome,"Caninde, Ceara",BR,CANINDE,CEARA,1,...,0,0,1,0,0,0,0,0,0,0
1,88d6c38491f774d5b9a6ea3fed2172,5f407e63c67d3c686eed,2022-04-12 19:09:46,Android,Chrome,"Campina Grande, Paraíba",BR,CAMPINA GRANDE,PARAÍBA,4,...,0,1,1,0,0,0,0,1,0,0
2,a4dc6fd225be473b9b0bef246d1cfc,d6505a3a5b07367b9134,2022-04-30 20:02:15,Android,Chrome,"Varzea Paulista, Sao Paulo",BR,VARZEA PAULISTA,SAO PAULO,1,...,0,0,0,0,0,0,1,0,0,0
3,2f4d961b5813c635cded4bd7469b52,e433eebec3fdfcc4cd7d,2022-03-10 21:57:05,Android,Chrome,"Garuva, Santa Catarina",BR,GARUVA,SANTA CATARINA,1,...,0,0,1,0,0,0,0,0,0,0
4,547e74c6665407ead0d4225e79519b,10166efe1d3a0f374fd7,2022-06-15 20:08:13,Android,Chrome,"Lago da Pedra, Maranhao",BR,LAGO DA PEDRA,MARANHAO,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371833,0d52256cf5ba59d6c236ea8455e17b,7022b90b9e3a9cc8e55c,2022-03-26 10:23:53,Android,Chrome,"Canoas, Rio Grande do Sul",BR,CANOAS,RIO GRANDE DO SUL,4,...,0,0,1,0,0,0,0,0,0,0
371834,4fab4ed9cfbc3605f79e201dba8bea,1d786782987b9c7037cf,2022-04-28 15:47:50,Android,Chrome,"Nova Iguaçu, Rio de Janeiro",BR,NOVA IGUAÇU,RIO DE JANEIRO,1,...,0,0,1,0,0,0,0,0,0,0
371835,b98435a4a17fd45f622738ec6c4ee5,ec925d86b78d4537a8f1,2022-06-20 22:59:07,Android,Chrome,"Santo André, Sao Paulo",BR,SANTO ANDRÉ,SAO PAULO,1,...,0,0,1,1,0,0,0,0,0,0
371836,5f43bf6055546a7ca980490fc9ac80,b6c0b82cc271fd1203fc,2022-06-15 09:13:15,Windows,Chrome,"São Paulo, Sao Paulo",BR,SÃO PAULO,SAO PAULO,7,...,0,0,0,0,0,0,0,0,1,0


In [140]:
# train the pipeline
price_pipe.fit(X_train, y_train)

C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\venv\lib\site-packages\feature_engine\encoding\rare_label.py:178: UserWarning: The number of unique categories for variable locale_x is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\venv\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[01:06:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




Pipeline(steps=[('missing_imputation',
                 CategoricalImputer(variables=['cidade_x', 'estado_x',
                                               'efeitos_desejados'])),
                ('frequent_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=['navegador', 'utm_source_x'])),
                ('rare_label_encoder',
                 RareLabelEncoder(n_categories=1, tol=0.01,
                                  variables=['sistema_operacional', 'navegador',
                                             'locale_x', 'cidad...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.1, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=3, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [141]:
pred_train = price_pipe.predict(X_train)

C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\venv\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [142]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
#
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# scores = cross_val_score(price_pipe, X_train[feature_selected], y_train, scoring='f1_micro', cv=cv, n_jobs=-1)

In [143]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, pred_train)

array([[240838,   7054],
       [ 91808,  32138]], dtype=int64)

In [144]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

print(metrics.classification_report(y_train, pred_train))
print('ROC: %.3f' % metrics.roc_auc_score(y_train, pred_train, average='micro'))

              precision    recall  f1-score   support

         0.0       0.72      0.97      0.83    247892
         1.0       0.82      0.26      0.39    123946

    accuracy                           0.73    371838
   macro avg       0.77      0.62      0.61    371838
weighted avg       0.76      0.73      0.68    371838

ROC: 0.615


In [145]:
pred_test = price_pipe.predict(X_test)


C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\venv\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [146]:
from sklearn import metrics

print(metrics.classification_report(y_test, pred_test))

              precision    recall  f1-score   support

         0.0       0.72      0.97      0.83     61974
         1.0       0.82      0.26      0.40     30987

    accuracy                           0.73     92961
   macro avg       0.77      0.62      0.61     92961
weighted avg       0.76      0.73      0.69     92961



In [147]:
from sklearn import metrics


print('ROC: %.3f' % metrics.roc_auc_score(y_test, pred_test, average='micro'))

ROC: 0.617


In [148]:
# #GridSearchCV
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RepeatedStratifiedKFold
#
#
# grid = GridSearchCV(price_pipe, param_grid={'xgboost__n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}, scoring='f1_micro', cv=cv, n_jobs=-1)
# grid.fit(X_train, y_train)


In [149]:
# grid.best_params_
# #

In [150]:
# #feature importance
# importance = grid.best_estimator_.steps[-1][1].feature_importances_
# importance = pd.DataFrame(importance, index=X_train.columns, columns=['importance'])
# importance.sort_values(by='importance', ascending=False, inplace=True)
# importance.plot(kind='barh')


In [151]:
#save model
import joblib

joblib.dump(price_pipe, r'C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\app\models\price_pipe.pkl')

['C:\\Users\\cassio.reis_saudeid\\Desktop\\CASSIO\\leads_classification\\app\\models\\price_pipe.pkl']

In [152]:
#load model
price_pipe = joblib.load(r'C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\app\models\price_pipe.pkl')

In [153]:
pred_test = price_pipe.predict(X_test)

C:\Users\cassio.reis_saudeid\Desktop\CASSIO\leads_classification\venv\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [154]:
print('ROC: %.3f' % metrics.roc_auc_score(y_test, pred_test, average='micro'))

ROC: 0.617


In [155]:
X_train.head(1).to_dict('records')

[{'id_form': '6d768849518ef901b121196cb158c0',
  'id_cliente': 'b15d5448c08b5cb6ee98',
  'data': '2022-03-12 07:48:03',
  'sistema_operacional': 'Android',
  'navegador': 'Chrome',
  'localizacao': 'Caninde, Ceara',
  'locale_x': 'BR',
  'cidade_x': 'CANINDE',
  'estado_x': ' CEARA',
  'qtde_lavagem_semana': 1,
  'utm_source_x': 'google',
  'tipo_cabelo': 'Tipo 4',
  'comprimento': 'Muito curto',
  'tipo_fios': 'Rebeldes',
  'procedimentos': 'Todos acima',
  'tempo_procedimento': 'Há mais de 1 mês',
  'dieta': 'Celíaca',
  'atividade_fisica': 'Raramente',
  'frequencia_estresse': 'Uma vez por mês',
  'faixa_etaria': '25 a 34',
  'efeitos_desejados': 'Efeito 4',
  'fragancia': 'Eucalipto',
  'id_pedido': nan,
  'data_pedido': nan,
  'status_pagamento': nan,
  'tipo_pagamento': nan,
  'valor_total': nan,
  'valor_desconto': nan,
  'frete_gratis': nan,
  'cidade_y': nan,
  'estado_y': nan,
  'locale_y': nan,
  'utm_source_y': nan,
  'Característica1': 0,
  'Característica10': 0,
  'Caract

In [156]:
df.head(1).to_dict('records')

[{'id_form': 'c7d453c9c01223a42968d9fe136047',
  'id_cliente': 'd2620a3e6f00a6dc6e88',
  'data': '2022-01-03 15:42:22',
  'sistema_operacional': 'iOS',
  'navegador': 'Safari',
  'localizacao': 'São Paulo, Sao Paulo',
  'locale_x': 'BR',
  'cidade_x': 'SÃO PAULO',
  'estado_x': ' SAO PAULO',
  'qtde_lavagem_semana': 4,
  'utm_source_x': nan,
  'tipo_cabelo': 'Tipo 2',
  'comprimento': 'Raíz exposta',
  'tipo_fios': 'Pontas duplas',
  'procedimentos': 'Nenhum',
  'tempo_procedimento': 'Há 2 semanas',
  'dieta': 'Celíaca',
  'atividade_fisica': 'Frequente',
  'frequencia_estresse': 'Nunca',
  'faixa_etaria': '25 a 34',
  'efeitos_desejados': 'Efeito 7',
  'fragancia': 'Rosas',
  'id_pedido': '5d2301cb6bf9d7c',
  'data_pedido': '2022-01-03 15:47:35.000000',
  'status_pagamento': 'Entregue',
  'tipo_pagamento': 'CARTÃO',
  'valor_total': 124.03,
  'valor_desconto': 0.0,
  'frete_gratis': False,
  'cidade_y': 'São Paulo',
  'estado_y': 'SP',
  'locale_y': 'BR',
  'utm_source_y': nan,
  'tar

In [157]:
X_train.columns

Index(['id_form', 'id_cliente', 'data', 'sistema_operacional', 'navegador',
       'localizacao', 'locale_x', 'cidade_x', 'estado_x',
       'qtde_lavagem_semana', 'utm_source_x', 'tipo_cabelo', 'comprimento',
       'tipo_fios', 'procedimentos', 'tempo_procedimento', 'dieta',
       'atividade_fisica', 'frequencia_estresse', 'faixa_etaria',
       'efeitos_desejados', 'fragancia', 'id_pedido', 'data_pedido',
       'status_pagamento', 'tipo_pagamento', 'valor_total', 'valor_desconto',
       'frete_gratis', 'cidade_y', 'estado_y', 'locale_y', 'utm_source_y',
       'Característica1', 'Característica10', 'Característica2',
       'Característica3', 'Característica4', 'Característica5',
       'Característica6', 'Característica7', 'Característica8',
       'Característica9', 'N/C'],
      dtype='object')

In [158]:
len(X_train.columns)

44

In [162]:
lst = ['id_form','id_cliente','data','sistema_operacional','navegador','localizacao','locale_x','cidade_x','estado_x','qtde_lavagem_semana','utm_source_x','tipo_cabelo','comprimento','tipo_fios','procedimentos','tempo_procedimento','dieta','atividade_fisica','frequencia_estresse','faixa_etaria','efeitos_desejados','fragancia','id_pedido','data_pedido','status_pagamento','tipo_pagamento','valor_total','valor_desconto','frete_gratis','cidade_y','estado_y','locale_y','utm_source_y','Característica1','Característica,1','Característica,2','Característica,3','Característica,4','Característica,5','Característica,6','Característica,7','Característica,8','Característica,9','Característica,10','N/C']

In [165]:
set(lst) - set(X_train.columns)

{'Característica,1',
 'Característica,10',
 'Característica,2',
 'Característica,3',
 'Característica,4',
 'Característica,5',
 'Característica,6',
 'Característica,7',
 'Característica,8',
 'Característica,9'}